Michelle 2/28/19

Replacement of Pigeon.jl w/o ROS requirements

Suitable for Collision Avoidance Controller using HJI severity

In [ ]:
using LinearAlgebra
using Distributed
using StaticArrays
using DifferentialDynamicsModels
using LinearDynamicsModels
using ForwardDiff
using Interpolations
using OSQP.MathOptInterfaceOSQP
using OSQP
import MathOptInterface
const MOI = MathOptInterface
using Parametron
using JLD2
import StaticArrays: SUnitRange
import DifferentialDynamicsModels: mod2piF, adiff
import Interpolations: GriddedInterpolation, Extrapolation
Parametron.Parameter(A::AbstractArray, model) = Parameter(identity, A, model)

include("math.jl")
include("vehicles.jl")
include("vehicle_dynamics.jl")
include("HJI_computation.jl")
include("HJI_Human.jl")
include("trajectories.jl")
include("model_predictive_control.jl")
# include("decoupled_lat_long.jl")
# include("coupled_lat_long.jl")
include("joint_traj_tracking.jl")
include("model_predictive_control.jl")
include("decoupled_lat_long.jl")
include("coupled_lat_long.jl")

const HJI_cache = HJICache(joinpath(@__DIR__, "../deps/BicycleCAvoid.jld2"));
const HJI_cache_Human = load_HJICache_Human(joinpath(@__DIR__, "../deps/HJIValue_Human_MinD.jld2"));

In [ ]:
trajectory = straight_trajectory(30., 5.) #(len, vel)

# Construct Coupled MPC --> trajectory
X1CMPC = CoupledTrajectoryTrackingMPC(X1(), trajectory, N_short=5, N_long=10)
X1CMPC.current_state = BicycleState(0., 0., 0., 9., 0., 0.) #(X, Y, th, Ux, Uy, yawrate)
X1CMPC.current_control = BicycleControl(0., 0., 0.) #(δ, Fxf, Fxr)

robotMPC = X1CMPC

# Construct Decoupled MPC --> HUMAN
robotMPC.other_car_state = SimpleCarState(5., -3.6, deg2rad(10), 4.) #(X, Y, th, V)

In [ ]:
# Load HJI Cache
robotMPC.HJI_cache_Human = HJI_cache_Human;

# Construct relative states
relative_state = HJIRelativeState_Human(robotMPC.current_state, robotMPC.other_car_state) #(ΔX, ΔY, θr, θh Ux)

# Extract Value
V, ∇V = robotMPC.HJI_cache_Human[relative_state]

In [ ]:
_, _, t = path_coordinates(robotMPC.trajectory, robotMPC.current_state)
compute_time_steps!(robotMPC, t)
compute_linearization_nodes!(robotMPC)
update_QP!(robotMPC)
solve!(robotMPC)

In [ ]:
s, e, _ = path_coordinates(robotMPC.trajectory, robotMPC.current_state)
u_next = get_next_control(robotMPC)

In [ ]:
qs, xs, us, ps = simulate(robotMPC, robotMPC.current_state, [0., 0., 0.])

In [ ]:
qs

In [ ]:
us

In [ ]:
ps